In [1]:
#No56(適合率，再現率，F1スコアの計測)
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
drive.mount('/content/drive')
!wget https://archive.ics.uci.edu/static/public/359/news+aggregator.zip
!unzip news+aggregator.zip
df = pd.read_csv("./newsCorpora.csv",sep="\t",header=None,names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])
#「PUBLISHER」の行から、”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
#isinはあくまでbool値を返すことに注意(Trueが抽出される)
df = df[df["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
#「TITLE」と「CATEGORY」の列を抽出する．
df = df[["TITLE", "CATEGORY"]]
#学習、検証、評価データに分割する(分割したいもの、割合、shuffleはTrueがデフォルト)
#まず0.9:0.1で分ける
train, test = train_test_split(df, test_size=0.2)
#次に0.2を半分にする(検証、評価データを0.1にする)
test, valid = train_test_split(test, test_size=0.5)

Mounted at /content/drive
--2024-06-07 05:40:22--  https://archive.ics.uci.edu/static/public/359/news+aggregator.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘news+aggregator.zip’

news+aggregator.zip     [     <=>            ]  27.87M  26.9MB/s    in 1.0s    

2024-06-07 05:40:23 (26.9 MB/s) - ‘news+aggregator.zip’ saved [29224203]

Archive:  news+aggregator.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [2]:
def preprosessing(text):
    #string.punctuation 「'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'」のこと
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    #maketransで作成したtableを文字列を変換する
    text = text.translate(table)
    #小文字にする
    text = text.lower()
    #正規表現を利用するために、compileを用いてpatternを作成する
    pattern = re.compile('[0-9]+')
    #正規表現にマッチした部分に0を代入
    text = re.sub(pattern, '0', text)

    return text

#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
#もとのindexを削除
df.reset_index(drop=True, inplace=True)

df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x))
#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
df.reset_index(drop=True, inplace=True)
#map関数　シーケンスの構成要素すべてに対して、ある関数の処理を行わせるという高階関数
#lambda関数を用いて、xを引数として、preprosessing関数を呼び出す
df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x))
vec_tfidf = TfidfVectorizer() #TfidfVectorizerのインスタンス生成
data = vec_tfidf.fit_transform(df['TITLE'])
data = pd.DataFrame(data.toarray(), columns = vec_tfidf.get_feature_names_out())
#整数除算をして、dataを分割する。
split_point1 = int(len(data)//3)
split_point2 = int(split_point1 * 2)
#学習、検証、評価データ
x_train = data[:split_point1]
x_valid = data[split_point1:split_point2]
x_test = data[split_point2:]
#学習、検証、評価等別
y_data = df['CATEGORY']
y_train = y_data[:split_point1]
y_valid = y_data[split_point1:split_point2]
y_test = y_data[split_point2:]

In [3]:
from sklearn.linear_model import LogisticRegression
#ロジスティック回帰モデルのインスタンス生成
model = LogisticRegression()
#訓練の実行(学習データ、ラベル(正解))
model.fit(x_train, y_train)

LogisticRegression()

In [4]:
# 正解率計算用のメソッド
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_valid)
#訓練データとテストデータを用いて予測する
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

In [7]:
#適合率　正例と分類されたサンプルのうち、実際に正例であるサンプルの割合
#再現率　本来は正例と判別すべきサンプルのうち、正しく正例であると判別できたサンプルの割合
#F1スコア　適合率と再現率の調和平均
#マクロ平均　カテゴリごとに評価指標を求め、全体の評価指標を平均する方法
#マイクロ平均 混合行列全体で TP、FP、FN を算出して、適合率、再現率、F値を計算する方法

#適合率，再現率，F1スコアを計測するメゾット
from sklearn.metrics import precision_score, recall_score, f1_score

def metrics(y_data, y_pred, ave=None):
  precision_sco = precision_score(y_data, y_pred, average=ave)
  recall_sco = recall_score(y_data, y_pred, average=ave)
  f1_sco = f1_score(y_data, y_pred, average=ave)
  form = "適合率：{}\n再現率：{}\nF１：{}\n".format(precision_sco, recall_sco, f1_sco)
  return form

print(f"【カテゴリ順】{model.classes_}\n\n{metrics(y_test, y_test_pred)}")
print("【マクロ平均】\n", metrics(y_test, y_test_pred, "macro"))
print("【マイクロ平均】\n", metrics(y_test, y_test_pred, "micro"))

【カテゴリ順】['b' 'e' 'm' 't']

適合率：[0.85513078 0.81273063 0.95774648 0.88235294]
再現率：[0.92744135 0.97834536 0.22591362 0.38011696]
F１：[0.88981942 0.88788108 0.3655914  0.53133515]

【マクロ平均】
 適合率：0.8769902080160582
再現率：0.6279543242467237
F１：0.6686567612676884

【マイクロ平均】
 適合率：0.8374550359712231
再現率：0.8374550359712231
F１：0.8374550359712231

